In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import copy

In [2]:
import mojimoji
import neologdn
import MeCab
def normalize_text(text):
    result = mojimoji.zen_to_han(text, kana=False)
    result = neologdn.normalize(result)
    return result


def text_to_words(text):
    m = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    m.parse('')
    #neologdnにより正規化処理をする。
    text = normalize_text(text)
    m_text = m.parse(text)
    basic_words = []
    #mecabの出力結果を単語ごとにリスト化
    m_text = m_text.split('\n')
    for row in m_text:
        #Tab区切りで形態素、その品詞等の内容と分かれているので単語部のみ取得
        word = row.split("\t")[0]
        #最終行はEOS
        if word == 'EOS':
            break
        else:
            pos = row.split('\t')[1]
            slice_ = pos.split(',')
            #品詞を取得する
            parts = slice_[0]
            if parts == '記号':
                if word != '。':
                    continue

                #読点のみ残す
                basic_words.append(word)
            #活用語の場合は活用指定ない原型を取得する。
            elif slice_[0] in ('形容詞', '動詞'):
                    basic_words.append(slice_[-3])

            #活用しない語についてはそのままの語を取得する
            elif slice_[0] in ('名詞', '副詞'):
                basic_words.append(word)

    basic_words = ' '.join(basic_words)
    return basic_words

In [3]:
ex_str1 = '日本語難しいね。日本語って単語にも分かれていないし面倒だわ〜。嫌いよ〜、本当に嫌いだわ。'
print(text_to_words(ex_str1))

日本語 難しい 。 日本語 単語 分かれる いる 面倒 。 嫌い 本当に 嫌い 。


In [4]:
df_train = pd.read_csv('./train.csv', header=0, sep='\t')
df_test = pd.read_csv('./test.csv', header=0, sep='\t')
print('train_data')
display(df_train.head())
print('test_data')
display(df_test.head())
x_train_text, y_train = [], []
x_test_text, y_test = [], []

train_flg = True
for df in (df_train, df_test):
    houses, committee, date = '', '', ''
    speech_text = []
    for item in tqdm_notebook(df.iterrows()):
        if  (houses == item[1]['houses'] and committee == item[1]['committee'] and date == item[1]['date']) :
            speech_text.append(copy.copy(item[1]['speech_text']))
            continue
        
        if train_flg:
            x_train_text.append(' '.join(copy.copy(speech_text)))
            y_train.append(copy.copy(committee))
            
        else:
            x_test_text.append(' '.join(copy.copy(speech_text)))
            y_test.append(copy.copy(committee))
            
        speech_text.clear()
        houses = item[1]['houses']
        committee = item[1]['committee']
        date = item[1]['date']
    
    train_flg= False

x_train_text.pop(0)
y_train.pop(0)
x_test_text.pop(0)
y_test.pop(0)
del df_train
del df_test

train_data


,houses,committee,speaker,date,speech_order,speech_text
0,衆議院,本会議,河野洋平,20080111,1,これより会議を開きます。◇
1,衆議院,本会議,河野洋平,20080111,2,本日、参議院から、本院送付のテロ対策海上阻止活動に対する補給支援活動の実施に関する特別措置法...
2,衆議院,本会議,河野洋平,20080111,3,大島理森君外百三名から、憲法第五十九条第二項に基づき、テロ対策海上阻止活動に対する補給支援活...
3,衆議院,本会議,仙谷由人,20080111,4,民主党の仙谷由人でございます。私は、民主党・無所属クラブを代表して、内閣提出のいわゆるテロ新...
4,衆議院,本会議,河野洋平,20080111,5,小坂憲次君。〔小坂憲次君登壇〕


test_data


,houses,committee,speaker,date,speech_order,speech_text
0,衆議院,議院運営委員会,古屋圭司,20180118,1,これより会議を開きます。まず、庶務小委員長から報告のため発言を求められておりますので、これを...
1,衆議院,議院運営委員会,石田真敏,20180118,2,平成三十年度の衆議院歳出予算の要求について、庶務小委員会における審議の経過及び結果について御...
2,衆議院,議院運営委員会,古屋圭司,20180118,3,この際、発言を求められておりますので、これを許します。塩川鉄也君。
3,衆議院,議院運営委員会,塩川鉄也,20180118,4,二〇一八年度の本院予算について意見を述べます。以下の理由により、本院予算に反対を表明します。...
4,衆議院,議院運営委員会,古屋圭司,20180118,5,それでは、平成三十年度本院歳出予算要求の件につきましては、小委員長の報告のとおり決定するに賛...


In [5]:
print(len(x_train_text))
print(len(y_train))
print(len(x_test_text))
print(len(y_test))

x_train_text = [text_to_words(x) for x in tqdm_notebook(x_train_text)]
x_test_text = [text_to_words(x) for x in tqdm_notebook(x_test_text)]

#便宜上、ラベルをIDに置き換えておく。
classify_dict = {'本会議':0, '内閣委員会':1, '総務委員会':2, '法務委員会':3, '外務委員会':4, '財務金融委員会':5, '文部科学委員会':6, '厚生労働委員会':7, '農林水産委員会':8, \
                '経済産業委員会':9, '国土交通委員会':10, '環境委員会':11, '安全保障委員会':12, '予算委員会':13, '決算行政監視委員会':14, '議院運営委員会':15}

y_train = [classify_dict[x] for x in y_train]
y_test = [classify_dict[x] for x in y_test]

7442
7442
821
821


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

corpus = x_train_text + x_test_text
train_size= len(x_train_text)

cv = CountVectorizer()
wc = cv.fit_transform(corpus)
ttf = TfidfTransformer()
tfidf = ttf.fit_transform(wc)

print(tfidf.shape)
x_train = tfidf[:train_size,:]
x_test = tfidf[train_size:,:]
print(x_train.shape)
print(x_test.shape)

(8263, 149818)
(7442, 149818)
(821, 149818)


In [7]:
ex_corpus = ['日本語 難しい ', '日本語 単語 分かれる いる 面倒 ', '嫌い 本当に 嫌い ']
ex_cv = CountVectorizer()
ex_wc = ex_cv.fit_transform(ex_corpus)
ex_ttf = TfidfTransformer()
ex_tfidf = ex_ttf.fit_transform(ex_wc)

#get_feature_names()で列がどの単語に対応しているか確認できる
display(pd.DataFrame(ex_tfidf.toarray(), columns=ex_cv.get_feature_names(), index=ex_corpus))

,いる,分かれる,単語,嫌い,日本語,本当に,難しい,面倒
日本語 難しい,0.000000,0.000000,0.000000,0.000000,0.605349,0.000000,0.795961,0.000000
日本語 単語 分かれる いる 面倒,0.467351,0.467351,0.467351,0.000000,0.355432,0.000000,0.000000,0.467351
嫌い 本当に 嫌い,0.000000,0.000000,0.000000,0.894427,0.000000,0.447214,0.000000,0.000000


In [38]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.001)
clf.fit(x_train, y_train)
test_predict = clf.predict(x_test)

In [39]:
#正解数をカウント
cnt = 0
#予想と結果をプロットするための行列
y_matrix = np.zeros((len(classify_dict), len(classify_dict)))
for idx, y in enumerate(y_test):
    if y == test_predict[idx]:
        cnt +=1
    
    y_matrix[y, test_predict[idx]] += 1


In [40]:
print('accuracy: %f' % (cnt/len(y_test)))
class_list = [x for x in classify_dict.keys()]
class_index = ['Actually_%s' % x for x in classify_dict]
class_col = ['Predict_%s' % x for x in classify_dict]
display(pd.DataFrame(y_matrix, index=class_index, columns=class_col, dtype=int))

accuracy: 0.834348


,Predict_本会議,Predict_内閣委員会,Predict_総務委員会,Predict_法務委員会,Predict_外務委員会,Predict_財務金融委員会,Predict_文部科学委員会,Predict_厚生労働委員会,Predict_農林水産委員会,Predict_経済産業委員会,Predict_国土交通委員会,Predict_環境委員会,Predict_安全保障委員会,Predict_予算委員会,Predict_決算行政監視委員会,Predict_議院運営委員会
Actually_本会議,69,3,1,4,0,1,2,8,8,2,2,3,0,0,0,2
Actually_内閣委員会,0,51,1,0,0,0,0,5,7,0,1,0,0,6,0,5
Actually_総務委員会,0,1,33,0,0,0,0,0,0,0,0,0,0,0,0,2
Actually_法務委員会,0,1,0,61,0,0,0,0,0,0,0,0,0,0,0,1
Actually_外務委員会,0,0,0,0,18,0,0,0,1,0,0,1,0,0,2,0
Actually_財務金融委員会,0,0,0,0,0,12,0,0,0,0,1,0,0,7,0,0
Actually_文部科学委員会,0,1,0,0,0,0,23,0,0,2,0,0,0,0,0,0
Actually_厚生労働委員会,0,0,5,0,0,0,0,74,0,0,1,0,0,0,0,1
Actually_農林水産委員会,0,0,0,0,1,0,0,1,64,0,0,0,0,0,0,3
Actually_経済産業委員会,0,1,0,0,0,0,0,0,0,32,0,3,0,0,0,2
